In [1]:
from itertools import repeat
import pickle5 as pickle
import pandas as pd
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
import rioxarray as rio
import rasterio
import numpy as np
import geopandas as gpd
from multiprocessing import Pool
import os
from rasterio.merge import merge
from itertools import product
from glob import glob
from pathlib import Path


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
from omegaconf import DictConfig,OmegaConf
conf = OmegaConf.load('config/config.yaml')
pic_=conf['config']['pic_']

In [3]:
grid = gpd.read_file("gs://bps-gcp-bucket/MLST2023/sample/Grid_Ekoreg_2022.gpkg").query('kdPIC=='+'\"'+pic_+'\"')

In [4]:
grid.columns

Index(['left', 'top', 'right', 'bottom', 'ID_GRID', 'identify_1', 'layer',
       'path', 'Bentuklahan', 'Kd_BL', 'Kd_Pulau', 'Nm_Pulau', 'Ekoregion1',
       'Kd_Ekoreg', 'kdPIC_old', 'kdPIC', 'geometry'],
      dtype='object')

In [5]:
def run_data(idgrid):
    tif_name='ml_output/08_class_result/result/classified_'+str(idgrid)+'.tif'
    if Path(tif_name).is_file()==False:
        run='python3 08_Production.py '+str(idgrid)
        !{run}
    else:
        print(idgrid,' is already classified. skip the process.')

In [9]:
for i in grid.ID_GRID.tolist():
    run_data(i)

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(
SKIP RASTER TEMP CUTTING
SKIP TEMP CLASSIFICATION
ml_output/08_class_result/result/classified_ID-3000.tif Done


In [ ]:
#with Pool(28) as p:
#    p.map(run_data,grid.ID_GRID.tolist()[:1])